In [50]:
%load_ext lab_black
import train
import pprint

import argparse
import os
import time
import numpy as np

import torch
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

from data.dataset import SequencePairDataset
from model.encoder_decoder import EncoderDecoder
from evaluate import evaluate
from utils import to_np, trim_seqs

from tensorboardX import SummaryWriter
from tqdm import tqdm
from data import parser
from data import reader

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [51]:
model_name = "sample_model"
data_list = reader.read_files(
    "./st-data/base.csv", "./st-data/styled.csv", parser.get_word_list
)
print("[INFO] show sample data\n")
print("[non-styled sentence] -> [styled sentence]")
print(str(data_list[0][0]) + "->" + str(data_list[1][0]) + "\n")

[INFO] show sample data

[non-styled sentence] -> [styled sentence]
['おはよう', 'ござい', 'ます', '。']->['おはよう。']



In [63]:
parser_ = argparse.ArgumentParser(description="Parse training parameters")
parser_.add_argument(
    "--model_name",
    type=str,
    default="model",
    help=(
        "the name of subdirectory of ./model/ that"
        + "contains encoder and decoder model files"
    ),
)
parser_.add_argument(
    "--epochs", type=int, default=200, help="the number of epochs to train"
)
parser_.add_argument(
    "--use_cuda",
    action="store_true",
    default="True",
    help="flag indicating that cuda will be used",
)
parser_.add_argument(
    "--batch_size", type=int, default=200, help="number of examples in a batch"
)
parser_.add_argument(
    "--teacher_forcing_fraction",
    type=float,
    default=2.0,
    help="fraction of batches that will use teacher forcing during training",
)
parser_.add_argument(
    "--scheduled_teacher_forcing",
    action="store_true",
    default=True,
    help=(
        "Linearly decrease the teacher forcing fraction"
        + " from 1.0 to 0.0 over the specified number of epochs"
    ),
)
parser_.add_argument(
    "--keep_prob",
    type=float,
    default=1.0,
    help="Probability of keeping an element in the dropout step.",
)
parser_.add_argument(
    "--val_size", type=float, default=0.1, help="fraction of data to use for validation"
)
parser_.add_argument("--lr", type=float, default=0.0005, help="Learning rate.")
parser_.add_argument(
    "--vocab_limit",
    type=int,
    default=100,
    help="When creating a new Language object the vocab"
    "will be truncated to the most frequently"
    "occurring words in the training dataset.",
)
parser_.add_argument(
    "--hidden_size",
    type=int,
    default=8,
    help="The number of RNN units in the encoder. 2x this "
    "number of RNN units will be used in the decoder",
)
parser_.add_argument(
    "--embedding_size",
    type=int,
    default=16,
    help="Embedding size used in both encoder and decoder",
)
parser_.add_argument(
    "--max_length",
    type=int,
    default=30,
    help="Sequences will be padded or truncated to this size.",
)
args = parser_.parse_args(
    args=[
        "--lr",
        "0.01",
        "--model_name",
        "sample_model",
        "--epochs",
        "200",
        "--batch_size",
        "100",
    ]
)

In [64]:
if args.scheduled_teacher_forcing:
    schedule = np.arange(1.0, 0.0, -1.0 / args.epochs)
else:
    schedule = np.ones(args.epochs) * args.teacher_forcing_fraction

In [65]:
writer = SummaryWriter("./sample_logs/%s_%s" % (args.model_name, str(int(time.time()))))
train.main(
    args.model_name,
    data_list,
    args.use_cuda,
    args.batch_size,
    schedule,
    args.keep_prob,
    args.val_size,
    args.lr,
    args.vocab_limit,
    args.hidden_size,
    args.embedding_size,
    args.max_length,
    writer,
)

training model_name with use_cuda=sample_model, batch_size=True
teacher_forcing_schedule=[1.    0.995 0.99  0.985 0.98  0.975 0.97  0.965 0.96  0.955 0.95  0.945
 0.94  0.935 0.93  0.925 0.92  0.915 0.91  0.905 0.9   0.895 0.89  0.885
 0.88  0.875 0.87  0.865 0.86  0.855 0.85  0.845 0.84  0.835 0.83  0.825
 0.82  0.815 0.81  0.805 0.8   0.795 0.79  0.785 0.78  0.775 0.77  0.765
 0.76  0.755 0.75  0.745 0.74  0.735 0.73  0.725 0.72  0.715 0.71  0.705
 0.7   0.695 0.69  0.685 0.68  0.675 0.67  0.665 0.66  0.655 0.65  0.645
 0.64  0.635 0.63  0.625 0.62  0.615 0.61  0.605 0.6   0.595 0.59  0.585
 0.58  0.575 0.57  0.565 0.56  0.555 0.55  0.545 0.54  0.535 0.53  0.525
 0.52  0.515 0.51  0.505 0.5   0.495 0.49  0.485 0.48  0.475 0.47  0.465
 0.46  0.455 0.45  0.445 0.44  0.435 0.43  0.425 0.42  0.415 0.41  0.405
 0.4   0.395 0.39  0.385 0.38  0.375 0.37  0.365 0.36  0.355 0.35  0.345
 0.34  0.335 0.33  0.325 0.32  0.315 0.31  0.305 0.3   0.295 0.29  0.285
 0.28  0.275 0.27  0.265 0.26  0.25

100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.42it/s]


=> <SOS>。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
val loss: 1.3874995592442168
----------------------------------------------------------------------------------------------------
epoch 1


100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.96it/s]


=> <SOS>。。。。。。。。。。。。。。。。。。。。。。。。。。。。。
val loss: 1.5557423046846197
----------------------------------------------------------------------------------------------------
epoch 2


...

=> <SOS>おはようん<UNK>ね<UNK><UNK>なね<SOS><SOS>おはようそうだね<UNK><UNK><UNK>なね<SOS><SOS>おはようそうだね<UNK><UNK><UNK>な
val loss: 504.11497965406346
----------------------------------------------------------------------------------------------------
epoch 198


100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.97it/s]


=> <SOS>そう<UNK>ね<UNK><UNK>な<SOS><SOS>おはようそうだね<UNK><UNK>なよね<SOS><SOS>おはようそうだね<UNK><UNK><UNK>なね<SOS>
val loss: 201.71631114842154
----------------------------------------------------------------------------------------------------
epoch 199


100%|███████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.59it/s]


=> <SOS>そう<UNK>ね<UNK><UNK>な<SOS><SOS>おはようそうだね<UNK><UNK><UNK>なね<SOS><SOS>おはようそうだね<UNK><UNK><UNK>な<SOS><SOS>
val loss: 355.86660195774635
----------------------------------------------------------------------------------------------------

=><SOS>そう<UNK>ね<UNK><UNK><UNK>な<SOS><SOS>おはようそう<UNK>ね<UNK><UNK><UNK>な<SOS><SOS>おはようそう<UNK>ね<UNK><UNK><UNK>な<SOS><SOS>


=><SOS>今日は良い天気ね<EOS>


=><SOS>Quit<UNK>ねね<UNK><UNK>な<SOS><SOS>QuitQuit行くね<UNK><UNK><UNK>な<SOS><SOS>QuitQuit行くね<UNK><UNK><UNK>な<SOS><SOS>

